In [1]:
import json

In [6]:
with open("./dev_20240627/dev_tied_append.json", "r") as f:
    ques_sql_lis = json.load(f)

with open("./dev_20240627/dev_tables.json", "r") as f:
    table_lis = json.load(f)

In [7]:
len(ques_sql_lis), len(table_lis)

(42, 11)

In [9]:
print(ques_sql_lis[0].keys())
print(table_lis[0].keys())

dict_keys(['question_id', 'db_id', 'question', 'evidence', 'SQL', 'difficulty'])
dict_keys(['db_id', 'table_names_original', 'table_names', 'column_names_original', 'column_names', 'column_types', 'primary_keys', 'foreign_keys'])


In [10]:
table_lis[0]

{'db_id': 'debit_card_specializing',
 'table_names_original': ['customers',
  'gasstations',
  'products',
  'transactions_1k',
  'yearmonth'],
 'table_names': ['customers',
  'gas stations',
  'products',
  'transactions',
  'year and month'],
 'column_names_original': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'Segment'],
  [0, 'Currency'],
  [1, 'GasStationID'],
  [1, 'ChainID'],
  [1, 'Country'],
  [1, 'Segment'],
  [2, 'ProductID'],
  [2, 'Description'],
  [3, 'TransactionID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'CustomerID'],
  [3, 'CardID'],
  [3, 'GasStationID'],
  [3, 'ProductID'],
  [3, 'Amount'],
  [3, 'Price'],
  [4, 'CustomerID'],
  [4, 'Date'],
  [4, 'Consumption']],
 'column_names': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'client segment'],
  [0, 'Currency'],
  [1, 'Gas Station ID'],
  [1, 'Chain ID'],
  [1, 'Country'],
  [1, 'chain segment'],
  [2, 'Product ID'],
  [2, 'Description'],
  [3, 'Transaction ID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'Customer ID'],
  [3, 'Card I

In [21]:
import json

def parse_schema(schema):
    """
    Parses a database schema to extract relevant details like tables, columns, primary keys, and foreign keys.
    :param schema: The raw schema dictionary
    :return: A structured schema summary
    """
    # Initialize the structured output
    structured_schema = {
        "Database": schema["db_id"],
        "Tables": []
    }

    # Iterate through tables
    for i, table_name in enumerate(schema["table_names"]):
        table_data = {
            "Table Name": table_name,
            "Description": f"Stores information about {table_name}." if table_name else "No description available",
            "Columns": [],
            "Primary Keys": [],
            "Foreign Keys": []
        }

        # Extract column data
        for column_info in schema["column_names"]:
            table_index = column_info[0]  # Determine which table the column belongs to
            if table_index == i:
                column_name = column_info[1]
                column_type = schema["column_types"][schema["column_names"].index(column_info)]
                
                # Create column entry
                column_entry = {
                    "Column Name": column_name,
                    "Type": column_type
                }

                # Add column to the table
                table_data["Columns"].append(column_entry)

        # Handle Primary Keys for each table
        if i < len(schema["primary_keys"]):
            primary_keys = schema["primary_keys"][i]
            if isinstance(primary_keys, list):  # Multiple primary keys
                for pk_index in primary_keys:
                    table_data["Primary Keys"].append(table_data["Columns"][pk_index]["Column Name"])
            else:  # Single primary key
                table_data["Primary Keys"].append(table_data["Columns"][primary_keys]["Column Name"])

        # Handle Foreign Keys for each table
        if i < len(schema["foreign_keys"]):
            for fk in schema["foreign_keys"][i]:
                if isinstance(fk, list):  # Correctly handling foreign key pairs (column index, referenced column index)
                    foreign_key_column_index = fk[0]  # The column index in the current table
                    referenced_column_index = fk[1]  # The referenced column index from the referenced table

                    # Ensure we retrieve the correct column names for both the foreign key and the referenced column
                    if foreign_key_column_index < len(table_data["Columns"]):
                        foreign_key_column = table_data["Columns"][foreign_key_column_index]
                        referenced_column_name = schema["column_names"][referenced_column_index][1]
                        table_data["Foreign Keys"].append({
                            "Column": foreign_key_column["Column Name"],
                            "References": referenced_column_name
                        })
                    else:
                        print(f"Warning: Foreign key column index {foreign_key_column_index} out of range.")
                else:  # Single integer foreign key, no list (assuming it maps directly to column index)
                    if fk < len(table_data["Columns"]):
                        foreign_key_column = table_data["Columns"][fk]
                        table_data["Foreign Keys"].append({
                            "Column": foreign_key_column["Column Name"],
                            "References": "Unknown"
                        })
                    else:
                        print(f"Warning: Foreign key index {fk} out of range.")

        structured_schema["Tables"].append(table_data)

    return structured_schema


def format_schema(schema_data):
    """
    Converts schema data into a human-readable, structured format.
    :param schema_data: Parsed schema data
    :return: Formatted JSON string
    """
    return json.dumps(schema_data, indent=2)


# Example schema (this can be any schema you have)
schema = {
    "db_id": "debit_card_specializing",
    "table_names_original": ['customers', 'gasstations', 'products', 'transactions_1k', 'yearmonth'],
    "table_names": ['customers', 'gas stations', 'products', 'transactions', 'year and month'],
    "column_names_original": [
        [-1, '*'],
        [0, 'CustomerID'],
        [0, 'Segment'],
        [0, 'Currency'],
        [1, 'GasStationID'],
        [1, 'ChainID'],
        [1, 'Country'],
        [1, 'Segment'],
        [2, 'ProductID'],
        [2, 'Description'],
        [3, 'TransactionID'],
        [3, 'Date'],
        [3, 'Time'],
        [3, 'CustomerID'],
        [3, 'CardID'],
        [3, 'GasStationID'],
        [3, 'ProductID'],
        [3, 'Amount'],
        [3, 'Price'],
        [4, 'CustomerID'],
        [4, 'Date'],
        [4, 'Consumption']
    ],
    "column_names": [
        [-1, '*'],
        [0, 'CustomerID'],
        [0, 'client segment'],
        [0, 'Currency'],
        [1, 'Gas Station ID'],
        [1, 'Chain ID'],
        [1, 'Country'],
        [1, 'chain segment'],
        [2, 'Product ID'],
        [2, 'Description'],
        [3, 'Transaction ID'],
        [3, 'Date'],
        [3, 'Time'],
        [3, 'Customer ID'],
        [3, 'Card ID'],
        [3, 'Gas Station ID'],
        [3, 'Product ID'],
        [3, 'Amount'],
        [3, 'Price'],
        [4, 'Customer ID'],
        [4, 'Date'],
        [4, 'Consumption']
    ],
    "column_types": [
        'text', 'integer', 'text', 'text', 'integer', 'integer', 'text', 'text', 'integer', 'text', 'integer', 'date', 'text',
        'integer', 'integer', 'integer', 'integer', 'integer', 'real', 'integer', 'text', 'real'
    ],
    "primary_keys": [1, 4, 8, 10, [19, 20]],
    "foreign_keys": [[19, 1]]
}

# Parse the schema and format the result
structured_schema = parse_schema(schema)
formatted_schema = format_schema(structured_schema)

# Print the formatted schema
print(formatted_schema)

# Access the first table in the structured schema
first_table = structured_schema["Tables"][0]

# Print the first table in a formatted way
print("\nFirst table data:")
print(json.dumps(first_table, indent=2))


IndexError: list index out of range

In [12]:
table_lis[0]

{'db_id': 'debit_card_specializing',
 'table_names_original': ['customers',
  'gasstations',
  'products',
  'transactions_1k',
  'yearmonth'],
 'table_names': ['customers',
  'gas stations',
  'products',
  'transactions',
  'year and month'],
 'column_names_original': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'Segment'],
  [0, 'Currency'],
  [1, 'GasStationID'],
  [1, 'ChainID'],
  [1, 'Country'],
  [1, 'Segment'],
  [2, 'ProductID'],
  [2, 'Description'],
  [3, 'TransactionID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'CustomerID'],
  [3, 'CardID'],
  [3, 'GasStationID'],
  [3, 'ProductID'],
  [3, 'Amount'],
  [3, 'Price'],
  [4, 'CustomerID'],
  [4, 'Date'],
  [4, 'Consumption']],
 'column_names': [[-1, '*'],
  [0, 'CustomerID'],
  [0, 'client segment'],
  [0, 'Currency'],
  [1, 'Gas Station ID'],
  [1, 'Chain ID'],
  [1, 'Country'],
  [1, 'chain segment'],
  [2, 'Product ID'],
  [2, 'Description'],
  [3, 'Transaction ID'],
  [3, 'Date'],
  [3, 'Time'],
  [3, 'Customer ID'],
  [3, 'Card I

In [ ]:
for i in ques_sql_lis:
    ques = i["question"]
    sql = i["SQL"]
    db_id = i["db_id"]

    for j in table_lis:
        if db_id == j["db_id"]:

            tab_nm = j["table_names_original"]
            